In [ ]:
import xarray as xr
import rioxarray
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
token = open("../flask/application/static/.mapbox_token").read()
px.set_mapbox_access_token(token)

In [ ]:
df = pd.read_csv("../flask/application/static/farsite_lonlat_low.csv")
df.head()

In [ ]:
fig = px.scatter_mapbox(df, lat="y", lon="x", color="US_DEM", opacity=0.1)
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# based on https://plotly.com/python/dropdowns/ and https://plot.ly/~empet/15237
# set layout
layout = go.Layout(mapbox=dict(accesstoken=token))

# load data
data = []
# display_columns = ["US_210CBD", "US_210CBH", "US_210CC", "US_210CH", "US_210EVC", "US_210EVH", "US_210F40",
#                    "US_210FVC", "US_210FVH", "US_210FVT", "US_ASP", "US_DEM", "US_FDIST", "US_SLP"]
display_columns = ["US_DEM", "US_SLP"]
for column in display_columns:
        data.append(
            go.Scattergeo(lat=df["y"], lon=df["x"], customdata=df, marker_color=df[column], opacity=0.1, visible=True)
        )
data[0].visible = True

# Create button list
buttons = []
for i, item in enumerate(display_columns):
    visibility = [False] * len(display_columns)
    visibility[i] = True
    buttons.append(dict(
        args=["visible", visibility],
        label=item,
        method="restyle"
    ))

# Add mapbox and dropdown
layout.update(
    mapbox=dict(accesstoken=token),
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Add annotation
layout.update(
    annotations=[
        dict(text="Data Layer:", showarrow=False,
        x=0, y=1.085, yref="paper", align="left")
    ]
)

fig = go.Figure(data=data, layout=layout)
fig.show()